# Acknowledgement
### This Notebook has been realised owing to the tutorial series by Valerio Velardo. The playlist can be found at https://www.youtube.com/channel/UCZPFjMe1uRSirmSpznqvJfQ/playlists under the title "Deep Learning (for Audio) with python. His tutorial involved genre-classification among 10 classes. 
I modified his work for my mood-classification among 4 classes task. However, the deep learning architecture used was not influenced by his tutorial. His main influence has been in user defined functions for saving and loading data with preprocessing the data, and ploting the history curve. Valerio has been made aware of me using his tutorial for my project through LinkedIn. He even undertook my survey and had like a post regarding my project on LinkedIn.
The following ipynb is for the PMEmo dataset, which required an additional feature to understand the song title. 

# Load Google Drives

In [ ]:
## Use the account 
## Email: arvind.final.dissertation@gmail.com
## Password: final paper

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# 2nd Drive if necessary

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!sudo mkdir /content/drive1
!google-drive-ocamlfuse /content/drive1

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144579 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

# Import Libraries and define google drive paths

In [ ]:
import json
import os
import math
import librosa
import pandas as pd

# Input_path = '/content/drive/My Drive/DEAM'
# Output_path = "/content/drive/My Drive/DEAM JSON/"

Input_path = '/content/drive1/PMEmo in folders'
Output_path = "/content/drive/My Drive/PMEmo json/"


TRACK_LEN = 15 # measured in seconds
SAMPLE_RATE = 44100
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_LEN

# User defined function to store the json values

In [ ]:
def create_json(input_loc, output_loc, coeff = 13, FFT_samples=2048, hop=512, track_pieces=5):
    """ Creates a JSON file too store the MFCC values of corresponding tracks along with the track id, label, and mapping dictionary.
        The tracks are segregating as per their emotional labels. 
        
        :param input_loc (string): dataset route
        :param output_loc (string): route to the location where the json file should be stored
        :param coeff (integer): Mel coefficients for the MFFCs
        :param FFT_samples (integer): Number of samples to apply Fast Fourier Transform(FFT)
        :param hop (integer): The number of samples by which the window slides
        :param track_pieces (integer): The number of pieces a track is divided into
        :return:
        """
    name = "Sample rate:{}, MFCC coefficient:{}, Track Duration:{}, segments: {}, FFT_samples = {}.json".format(SAMPLE_RATE, coeff, TRACK_LEN, track_pieces, FFT_samples)
    final_loc = output_loc + name
    # dictionary to store mapping, labels, and MFCCs
    data = {
        "map": [],
        "label_encode": [],
        "mfcc_values": [],
        "musicId": []
    }

    samples_per_piece = int(SAMPLES_PER_TRACK / track_pieces)
    mfcc_per_piece = math.ceil(samples_per_piece / hop)

    # loop through all genre sub-folder
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(input_loc)):

        # ensure we're processing a genre sub-folder level
        if dirpath is not input_loc:

            # emotion label is stored in map
            label = dirpath.split("/")[-1]
            data["map"].append(label)
            print("\nEmotion Label: {}".format(label))

            # process all audio files in genre sub-dir
            for f in filenames:

		            # audio file are invoked
                file = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file, sr=SAMPLE_RATE)
                f = int(f.split(".mp3")[0])

                # process all segments of audio file
                for k in range(track_pieces):

                    # calculate start and finish sample for current segment
                    beginning = samples_per_piece * k
                    end = beginning + samples_per_piece

                    # extract MFCC
                    MFCC = librosa.feature.mfcc(signal[beginning:end], sample_rate, n_mfcc=coeff, n_fft=FFT_samples, hop_length=hop)
                    MFCC = MFCC.T

                    # store only mfcc feature with expected number of vectors
                    if len(MFCC) == mfcc_per_piece:
                        data["mfcc_values"].append(MFCC.tolist())
                        data["label_encode"].append(i-1)
                        data["musicId"].append(int(f))
                        print("{}, segment:{}, musicId: {}, label_code: {}".format(file, k+1, int(f), i-1))

    # save MFCCs to json file
    with open(final_loc, "w") as fp:
        json.dump(data, fp)

# Audio --> Json file containing MFFCs and other information 

In [ ]:
create_json(Input_path, Output_path, track_pieces = 20) #PMEmo 2019

Streaming output truncated to the last 5000 lines.
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:6, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:7, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:8, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:9, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:10, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:11, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:12, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:13, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:14, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:15, musicId: 236, label_code: 1
/content/drive1/PMEmo in folders/Sad/236.mp3, segment:16, musicId: 236, label_code: 1
/conten